# Demos: Lecture 9

## Demo 1: `qml.specs` revisited

In [12]:
import pennylane as qml
from pprint import pprint

In [3]:
n_bits = 3
n_work_wires = 2
special_string = '110'

dev = qml.device('default.qubit', wires=n_bits+1+n_work_wires)


def hadamard_transform(wires=None):
    for wire in wires:
        qml.Hadamard(wires=wire)

def oracle():
    qml.MultiControlledX(
        control_wires=range(n_bits), 
        wires=n_bits, 
        control_values=special_string,
        work_wires=range(n_bits+1, n_bits+1+n_work_wires)
    )
    
def diffusion():
    hadamard_transform(wires=range(n_bits))
    qml.MultiControlledX(
        control_wires=range(n_bits), 
        wires=n_bits, 
        control_values='0'*n_bits,
        work_wires=range(n_bits+1, n_bits+1+n_work_wires)
    )
    hadamard_transform(wires=range(n_bits))

In [4]:
@qml.qnode(dev, expansion_strategy='device')
def grover(num_steps=1):
    qml.PauliX(wires=n_bits)
    
    hadamard_transform(wires=range(n_bits+1))
        
    for _ in range(num_steps):
        oracle()
        diffusion()
        
    return qml.probs(wires=range(n_bits))    

In [5]:
print(qml.draw(grover)(num_steps=2))

 0: ──H─────╭C──H──╭C──H──╭C──H──╭C──H──╭┤ Probs 
 1: ──H─────├C──H──├C──H──├C──H──├C──H──├┤ Probs 
 2: ──H─────├C──H──├C──H──├C──H──├C──H──╰┤ Probs 
 3: ──X──H──╰X─────╰X─────╰X─────╰X──────┤       



In [6]:
pprint(qml.specs(grover)(num_steps=2))

{'depth': 10,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'device',
 'gate_sizes': defaultdict(<class 'int'>, {1: 17, 4: 4}),
 'gate_types': defaultdict(<class 'int'>,
                           {'Hadamard': 16,
                            'MultiControlledX': 4,
                            'PauliX': 1}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'autograd',
 'num_device_wires': 6,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 21,
 'num_trainable_params': 0,
 'num_used_wires': 4}


In [7]:
dev.operations.remove('MultiControlledX')

In [8]:
print(qml.draw(grover)(num_steps=2))

 0: ──H─────────╭C─────────╭C──H──X──────╭C─────────╭C──X──H──────╭C─────────╭C──H──X──────╭C─────────╭C──X──H──╭┤ Probs 
 1: ──H─────────├C─────────├C──H──X──────├C─────────├C──X──H──────├C─────────├C──H──X──────├C─────────├C──X──H──├┤ Probs 
 2: ──H──X──╭C──│───╭C──X──│───H──X──╭C──│───╭C──X──│───H──X──╭C──│───╭C──X──│───H──X──╭C──│───╭C──X──│───H─────╰┤ Probs 
 3: ──X──H──├X──│───├X─────│─────────├X──│───├X─────│─────────├X──│───├X─────│─────────├X──│───├X─────│──────────┤       
 4: ────────╰C──╰X──╰C─────╰X────────╰C──╰X──╰C─────╰X────────╰C──╰X──╰C─────╰X────────╰C──╰X──╰C─────╰X─────────┤       



In [9]:
pprint(qml.specs(grover)(num_steps=2))

{'depth': 26,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'device',
 'gate_sizes': defaultdict(<class 'int'>, {1: 33, 3: 16}),
 'gate_types': defaultdict(<class 'int'>,
                           {'Hadamard': 16,
                            'PauliX': 17,
                            'Toffoli': 16}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'autograd',
 'num_device_wires': 6,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 49,
 'num_trainable_params': 0,
 'num_used_wires': 5}


## Demo 2: quantum tapes

In [13]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def my_function():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.S(wires=1)
    qml.RX(0.3, wires=2)
    qml.RY(0.8, wires=3)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliX(1))

In [14]:
my_function.qtape

In [15]:
my_function()

tensor(0., requires_grad=True)

In [16]:
my_function.qtape

<JacobianTape: wires=[0, 1, 2, 3], params=0>

In [17]:
my_function.qtape.operations

[Hadamard(wires=[0]),
 CNOT(wires=[0, 1]),
 S(wires=[1]),
 RX(0.3, wires=[2]),
 RY(0.8, wires=[3]),
 CNOT(wires=[0, 1])]

In [18]:
my_function.qtape.measurements

[expval(PauliX(wires=[1]))]

## Demo 3: tape/qfunc transforms

In [19]:
@qml.qfunc_transform
def x_to_hzh(tape):
    for op in tape.operations + tape.measurements:
        if op.name == "PauliX":
            qml.Hadamard(wires=op.wires)
            qml.PauliZ(wires=op.wires)
            qml.Hadamard(wires=op.wires)
        else:
            qml.apply(op)

In [20]:
def some_pauli_x():
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    return qml.probs(wires=[0, 1])

In [21]:
dev = qml.device('default.qubit', wires=2)

some_qnode = qml.QNode(some_pauli_x, dev)

print(qml.draw(some_qnode)())

 0: ──X──╭┤ Probs 
 1: ──X──╰┤ Probs 



In [22]:
new_function = x_to_hzh(some_pauli_x)

In [23]:
new_qnode = qml.QNode(new_function, dev)

print(qml.draw(new_qnode)())

 0: ──H──Z──H──╭┤ Probs 
 1: ──H──Z──H──╰┤ Probs 



In [24]:
@qml.qfunc_transform
def z_to_hxh(tape):
    for op in tape.operations + tape.measurements:
        if op.name == "PauliZ":
            qml.Hadamard(wires=op.wires)
            qml.PauliX(wires=op.wires)
            qml.Hadamard(wires=op.wires)
        else:
            qml.apply(op)

In [25]:
@z_to_hxh
@x_to_hzh
def some_pauli_x():
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    return qml.probs(wires=[0, 1])

qnode = qml.QNode(some_pauli_x, dev)

In [26]:
print(qml.draw(qnode)())

 0: ──H──H──X──H──H──╭┤ Probs 
 1: ──H──H──X──H──H──╰┤ Probs 



## Demo 4: circuit compilation (small)

In [27]:
@qml.transforms.cancel_inverses
@z_to_hxh
@x_to_hzh
def some_pauli_x():
    qml.PauliX(wires=0)
    qml.PauliX(wires=1)
    return qml.probs(wires=[0, 1])

qnode = qml.QNode(some_pauli_x, dev)

In [28]:
print(qml.draw(qnode)())

 0: ──X──╭┤ Probs 
 1: ──X──╰┤ Probs 



## Demo 5: circuit compilation (large)

In [1]:
import numpy as np
import pennylane as qml

/opt/conda/envs/pennylane/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [18]:
dev = qml.device('default.qubit', wires=6)

weights = np.random.rand(len(dev.wires), 7)

def big_function():
    for wire in dev.wires:
        qml.Hadamard(wires=wire)
        
    qml.RandomLayers(weights, dev.wires, seed=0)
    
    return qml.state()

In [8]:
original_qnode = qml.QNode(big_function, dev, expansion_strategy='device')

In [5]:
print(qml.draw(original_qnode, expansion_strategy='device')())

 0: ──H──RZ(0.314)────────RY(0.796)─────────────────────────────╭C──╭C──────────────╭C───RZ(0.6)────RZ(0.0896)───RY(0.242)──────────────╭C──RX(0.0804)──RY(0.272)──────╭C─────────────────────────╭X───RY(0.115)─────────────────────────╭C───────────────────╭C────────────────────╭┤ State 
 1: ──H──────────────╭X──────────────╭X──RY(0.22)───╭C──────────│───╰X──────────╭C──│───╭C──────────RZ(0.778)───╭C───────────RX(0.724)──│───RZ(0.784)───RY(0.976)──╭X──│───╭C──────────RX(0.51)───│───────────────╭X──────────RX(0.223)──│───╭C──╭C───────────│─────────────────────├┤ State 
 2: ──H──RX(0.301)───│───╭X──────────│──────────────╰X──────────│────RY(0.48)───│───│───│───────────────────────╰X───────────RX(0.981)──│───RY(0.221)──────────────│───│───╰X─────────────────────│───────────────│──────────────────────│───│───╰X───────────╰X────────────────────├┤ State 
 3: ──H──RX(0.207)───│───╰C──────────│───RX(0.812)───RZ(0.118)──│───────────────│───│───│───────────────────────────────────────────────│─────

In [6]:
from pprint import pprint

In [9]:
pprint(qml.specs(original_qnode)())

{'depth': 24,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'device',
 'gate_sizes': defaultdict(<class 'int'>, {1: 48, 2: 22}),
 'gate_types': defaultdict(<class 'int'>,
                           {'CNOT': 22,
                            'Hadamard': 6,
                            'RX': 16,
                            'RY': 13,
                            'RZ': 13}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'autograd',
 'num_device_wires': 6,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 70,
 'num_trainable_params': 42,
 'num_used_wires': 6}


In [19]:
pipeline = [
    qml.transforms.cancel_inverses,
    qml.transforms.commute_controlled(direction="left"),
    qml.transforms.single_qubit_fusion(),
]

In [20]:
new_qnode = qml.QNode(
    qml.compile(pipeline=pipeline)(big_function),
    dev,
    expansion_strategy='device'
)
    

In [21]:
pprint(qml.specs(new_qnode)())

{'depth': 19,
 'device_name': 'default.qubit.autograd',
 'diff_method': 'best',
 'expansion_strategy': 'device',
 'gate_sizes': defaultdict(<class 'int'>, {1: 21, 2: 20}),
 'gate_types': defaultdict(<class 'int'>,
                           {'CNOT': 20,
                            'RY': 1,
                            'RZ': 1,
                            'Rot': 19}),
 'gradient_fn': 'backprop',
 'gradient_options': {},
 'interface': 'autograd',
 'num_device_wires': 6,
 'num_diagonalizing_gates': 0,
 'num_observables': 1,
 'num_operations': 41,
 'num_trainable_params': 0,
 'num_used_wires': 6}
